In [ ]:
!nvidia-smi

Wed May 10 03:19:26 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.85.12    Driver Version: 525.85.12    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   41C    P8     9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive

Mounted at /content/gdrive


In [ ]:
%cd /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook/
%ls

/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook
golfdb/                                  golfdb_xgg_inference_best.ipynb
golfdb_val_test_xgg_inference_tcn.ipynb  golfdb_xgg_inference_tcn.ipynb


In [ ]:
!git clone https://github.com/wmcnally/golfdb.git
%cd golfdb/
%ls

fatal: destination path 'golfdb' already exists and is not an empty directory.
/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/src/Notebook/golfdb
data/                             golfdb_tcn_fold5_iter6000_X.csv
dataloader_custom.py              golfdb_tcn_fold5_iter7000_X.csv
dataloader.py                     golfdb_tcn_fold5_iter8000_X.csv
eval.py                           golfdb_tcn_fold5_iter9000_X.csv
golfdb/                           golfdb_val_test_tcn_fold5_iter1000_X.csv
golfdb_tcn_fold4_iter10000_X.csv  MobileNetV2_hact.py
golfdb_tcn_fold4_iter1000_X.csv   mobilenet_v2.pth.tar
golfdb_tcn_fold4_iter2000_X.csv   MobileNetV2.py
golfdb_tcn_fold4_iter3000_X.csv   model_custom_hact.py
golfdb_tcn_fold4_iter4000_X.csv   model_custom.py
golfdb_tcn_fold4_iter5000_X.csv   model_custom_tcn.py
golfdb_tcn_fold4_iter6000_X.csv   model.py
golfdb_tcn_fold4_iter7000_X.csv   __pycache__/
golfdb_tcn_fold4_iter8000_X.csv   REA

## Import packages

In [ ]:
import argparse
import os
import cv2
import csv
import random
import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from eval import ToTensor, Normalize
from model_custom import EventDetector
import torch.nn.functional as F

## functions

In [ ]:
event_names = {
    0: 'HitFrame'
}


class SampleVideo(Dataset):
    def __init__(self, path, input_size=160, transform=None):
        self.path = path
        self.input_size = input_size
        self.transform = transform

    def __len__(self):
        return 1

    def __getitem__(self, idx):
        cap = cv2.VideoCapture(self.path)
        frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
        print('frame_count =', frame_count)
        frame_size = [cap.get(cv2.CAP_PROP_FRAME_HEIGHT), cap.get(cv2.CAP_PROP_FRAME_WIDTH)]
        ratio = self.input_size / max(frame_size)
        new_size = tuple([int(x * ratio) for x in frame_size])
        delta_w = self.input_size - new_size[1]
        delta_h = self.input_size - new_size[0]
        top, bottom = delta_h // 2, delta_h - (delta_h // 2)
        left, right = delta_w // 2, delta_w - (delta_w // 2)

        #print(new_size)
        # preprocess and return frames
        images = []
        for pos in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
            _, img = cap.read()

            img = img[:, 280:1280-280]
            resized = cv2.resize(img, (180, 180), interpolation=cv2.INTER_AREA)    ################################ 1. test-size ################################

            #resized = cv2.resize(img, (new_size[1], new_size[0]))
            b_img = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT,
                                       value=[0.406 * 255, 0.456 * 255, 0.485 * 255])  # ImageNet means (BGR)

            b_img_rgb = cv2.cvtColor(b_img, cv2.COLOR_BGR2RGB)
            images.append(b_img_rgb)
        cap.release()
        labels = np.zeros(len(images)) # only for compatibility with transforms
        sample = {'images': np.asarray(images), 'labels': np.asarray(labels)}
        if self.transform:
            sample = self.transform(sample)
        return sample    

In [ ]:
%ls /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/part1/val_test_xgg/    ################################ 2. val-folder ################################

00001_xgg.mp4  00081_xgg.mp4  00161_xgg.mp4  00241_xgg.mp4  00321_xgg.mp4
00002_xgg.mp4  00082_xgg.mp4  00162_xgg.mp4  00242_xgg.mp4  00322_xgg.mp4
00003_xgg.mp4  00083_xgg.mp4  00163_xgg.mp4  00243_xgg.mp4  00323_xgg.mp4
00004_xgg.mp4  00084_xgg.mp4  00164_xgg.mp4  00244_xgg.mp4  00324_xgg.mp4
00005_xgg.mp4  00085_xgg.mp4  00165_xgg.mp4  00245_xgg.mp4  00325_xgg.mp4
00006_xgg.mp4  00086_xgg.mp4  00166_xgg.mp4  00246_xgg.mp4  00326_xgg.mp4
00007_xgg.mp4  00087_xgg.mp4  00167_xgg.mp4  00247_xgg.mp4  00327_xgg.mp4
00008_xgg.mp4  00088_xgg.mp4  00168_xgg.mp4  00248_xgg.mp4  00328_xgg.mp4
00009_xgg.mp4  00089_xgg.mp4  00169_xgg.mp4  00249_xgg.mp4  00329_xgg.mp4
00010_xgg.mp4  00090_xgg.mp4  00170_xgg.mp4  00250_xgg.mp4  00330_xgg.mp4
00011_xgg.mp4  00091_xgg.mp4  00171_xgg.mp4  00251_xgg.mp4  00331_xgg.mp4
00012_xgg.mp4  00092_xgg.mp4  00172_xgg.mp4  00252_xgg.mp4  00332_xgg.mp4
00013_xgg.mp4  00093_xgg.mp4  00173_xgg.mp4  00253_xgg.mp4  00333_xgg.mp4
00014_xgg.mp4  00094_xgg.mp4  00174_xg

In [ ]:
seq_length = 64    ################################ 3. seq-length ################################


valPath = "/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/part1/val_test_xgg/"    ################################ 4. val-path ################################
valFolders = os.listdir(valPath)
valFolders = sorted(valFolders)

valVideoPath = []
valVideos = []
"""
for folder in valFolders:
    fileName = os.listdir(valPath + folder + '/')
    for fn in fileName:
        if fn[-1] == '4':
            videoPath = valPath + folder + '/' + fn
            valVideoPath.append(videoPath)
            valVideos.append(fn)
"""
for fn in valFolders:
    videoPath = valPath + fn
    valVideoPath.append(videoPath)
    valVideos.append(fn)

#print(valVideos)


predictList = []

data = []

for vp in valVideoPath:
    
    #vp = '/home/yuhsi/Badminton/data/part1/val/00004/00004.mp4'    # debug

    print('Preparing video: {}'.format(vp))

    ds = SampleVideo(vp, transform=transforms.Compose([ToTensor(),
                                Normalize([0.485, 0.456, 0.406],
                                        [0.229, 0.224, 0.225])]))

    dl = DataLoader(ds, batch_size=1, shuffle=False, drop_last=False)

    model = EventDetector(pretrain=True,
                        width_mult=1.,
                        lstm_layers=1,
                        lstm_hidden=256,
                        bidirectional=True,
                        dropout=False)

    try:
        save_dict = torch.load('/content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/weights/golfdb_G3/models/fold5_swingnet_3000.pth.tar')    ################################ 5. trained-weight ################################
    except:
        print("Model weights not found. Download model weights and place in 'models' folder. See README for instructions")

    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print('Using device:', device)
    model.load_state_dict(save_dict['model_state_dict'])
    model.to(device)
    model.eval()
    print("Loaded model weights")

    print('Testing...')
    for sample in dl:
        images = sample['images']
        # full samples do not fit into GPU memory so evaluate sample in 'seq_length' batches
        batch = 0
        while batch * seq_length < images.shape[1]:
            if (batch + 1) * seq_length > images.shape[1]:
                image_batch = images[:, batch * seq_length:, :, :, :]
            else:
                image_batch = images[:, batch * seq_length:(batch + 1) * seq_length, :, :, :]
            logits = model(image_batch.cuda())
            if batch == 0:
                probs = F.softmax(logits.data, dim=1).cpu().numpy()
            else:
                probs = np.append(probs, F.softmax(logits.data, dim=1).cpu().numpy(), 0)
            batch += 1


    probList = probs[:, 0]
    #print(probList)
    
    #events = np.argmax(probs, axis=0)[:-1]
    #print('Predicted event frames: {}'.format(events))

    eventList = []
    #thres = np.average(probList)
    higher_q = np.quantile(probList, 0.8, interpolation='higher')    ################################ 6. filter-thres ################################
    thres = higher_q
    #print(thres, higher_q)
    for pi in range(len(list(probList))):
        if list(probList)[pi] > thres:
            eventList.append([pi])
    print('Predicted event frames: {}'.format(eventList))

    # filter the events
    filteredList = [[]]
    for i in range(len(eventList)-1):
        if np.abs(eventList[i+1][0] - eventList[i][0]) == 1:
            filteredList[-1].append(eventList[i][0])
        else:
            filteredList[-1].append(eventList[i][0])
            filteredList.append([])
    print('filteredList =', filteredList)

    new_eventList = []
    for l in filteredList:
        if len(l) >= 3:    ################################ 7. filter-len ################################
            ll = []
            for i in range(len(l)):
                ll.append(probs[l[i]][0])
            e = l[np.argmax(ll)]
            #print(e)
            new_eventList.append([e])
    print('new_eventList =', new_eventList)


    eventList = new_eventList


    ss = 1
    vn_ = vp.split('/')[-1]
    vn = vn_.split('_')[0] + '.mp4'
    for ie in range(len(eventList)):
        #print(vn, eventList[ie][0])
        if ie == len(eventList) - 1:
            data.append([vn, ss, eventList[ie][0], 'X', 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'X'])
        else:
            data.append([vn, ss, eventList[ie][0], 'X', 
                        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 'X'])
        

        # visualize
        """
        cap = cv2.VideoCapture(vp)
        confidence = []
        events = eventList[ie]
        for i, e in enumerate(events):
            confidence.append(probs[e, i])
        print('Condifence: {}'.format([np.round(c, 3) for c in confidence]))

        for i, e in enumerate(events):
            cap.set(cv2.CAP_PROP_POS_FRAMES, e)
            _, img = cap.read()
            cv2.putText(img, '{:.3f}'.format(confidence[i]), (20, 20), cv2.FONT_HERSHEY_DUPLEX, 0.75, (0, 0, 255))
            cv2.imshow(event_names[i], img)
            cv2.waitKey(0)
            cv2.destroyAllWindows()
        """

        ss += 1
    #break


# write csv file
header = ['VideoName', 'ShotSeq', 'HitFrame', 'Hitter', 'RoundHead', 'Backhand', 'BallHeight',
          'LandingX', 'LandingY', 'HitterLocationX', 'HitterLocationY', 
          'DefenderLocationX', 'DefenderLocationY', 'BallType', 'Winner']

with open('golfdb_G3_fold5_iter3000_val_test_X.csv', 'w', encoding='UTF8', newline='') as f:    ################################ 8. output-name ################################
    writer = csv.writer(f)

    # write the header
    writer.writerow(header)

    # write multiple rows
    writer.writerows(data)

Preparing video: /content/gdrive/MyDrive/Teaching_Computer_to_Watch_Badminton_Matches_Taiwan_first_competition_combining_AI_and_sports/datasets/part1/val_test_xgg/00001_xgg.mp4
Using device: cuda
Loaded model weights
Testing...
frame_count = 1394


<ipython-input-8-dd7b44e28fd4>:88: DeprecationWarning: the `interpolation=` argument to quantile was renamed to `method=`, which has additional options.
Users of the modes 'nearest', 'lower', 'higher', or 'midpoint' are encouraged to review the method they. (Deprecated NumPy 1.22)
  higher_q = np.quantile(probList, 0.8, interpolation='higher')    ################################ 6. filter-thres ################################


Predicted event frames: [[45], [46], [54], [55], [56], [61], [62], [63], [64], [71], [72], [74], [75], [77], [78], [79], [80], [81], [82], [99], [100], [101], [102], [103], [104], [114], [127], [128], [129], [130], [152], [153], [154], [155], [156], [185], [186], [187], [188], [214], [215], [216], [224], [225], [240], [241], [242], [243], [244], [245], [246], [269], [270], [271], [272], [273], [274], [311], [312], [313], [314], [329], [330], [331], [332], [333], [334], [354], [355], [356], [357], [358], [359], [427], [428], [429], [430], [431], [432], [448], [449], [466], [467], [468], [491], [492], [493], [494], [495], [496], [497], [510], [511], [512], [513], [514], [515], [534], [535], [536], [537], [569], [570], [571], [572], [573], [574], [576], [577], [591], [592], [593], [594], [595], [596], [597], [622], [623], [624], [625], [626], [627], [660], [661], [662], [663], [664], [688], [689], [690], [691], [706], [707], [708], [709], [710], [711], [712], [713], [714], [735], [736], [